In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://dps.arkansas.gov/law-enforcement/arkansas-state-police/directors-office/asp-commission/asp-commission-meeting-minutes/"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
}

r = requests.get(url, headers=headers)
r.raise_for_status()

soup = BeautifulSoup(r.text, "html.parser")

data = []

# Loop through all headings like "2025 Meeting Minutes", "2024 Meeting Minutes", etc.
for h3 in soup.find_all("h3"):
    year_text = h3.get_text(strip=True)

    # The list is the next <ul> after the <h3>
    ul = h3.find_next("ul")
    if not ul:
        continue

    for li in ul.find_all("li"):
        a = li.find("a")
        if not a:
            continue

        data.append({
            "year_group": year_text,
            "meeting_date": a.get_text(strip=True),
            "pdf_url": a["href"] if a["href"].startswith("http") else "https://dps.arkansas.gov" + a["href"]
        })

df = pd.DataFrame(data)
df


,year_group,meeting_date,pdf_url
0,2025 Meeting Minutes,"September 11, 2025",https://dps.arkansas.gov/wp-content/uploads/AS...
1,2025 Meeting Minutes,"August 14, 2025",https://dps.arkansas.gov/wp-content/uploads/AS...
2,2025 Meeting Minutes,"July 10, 2025",https://dps.arkansas.gov/wp-content/uploads/AS...
3,2025 Meeting Minutes,"June 12, 2025",https://dps.arkansas.gov/wp-content/uploads/AS...
4,2025 Meeting Minutes,"May 8, 2025",https://dps.arkansas.gov/wp-content/uploads/AS...
...,...,...,...
165,2013 Meeting Minutes,"June 3, 2013",https://dps.arkansas.gov/wp-content/uploads/20...
166,2013 Meeting Minutes,"May 9, 2013",https://dps.arkansas.gov/wp-content/uploads/20...
167,2013 Meeting Minutes,"April 11, 2013",https://dps.arkansas.gov/wp-content/uploads/20...
168,2013 Meeting Minutes,"March 14, 2013",https://dps.arkansas.gov/wp-content/uploads/20...


In [11]:
#saving to csv 
df.to_csv("arkansas_meeting_minutes.csv", index=False)
